import 程式庫


In [1]:
from database import Database
from get_data import Data
from seperate_company import SeperateCompany
from plot import Plot
from backtest import Backtest
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
# 建立取得資料物件
data = Data()
# 建立資料分割物件
seperate_company = SeperateCompany()
# 建立畫圖物件
plotter = Plot()

In [ ]:
# 測試輸出財報資料    
# factor_dict = data.get('report:EPS,ROE,PB,PS,PE')
factor_dict = data.get('report:EPS,ROE')

In [ ]:
# 根據隨入的因子，將公司分成quantile組
# 回測所有quantile
all_result_dict = seperate_company.backtest_all_quantile(factor_dict,quantile=4, start_date="2013-01-01", end_date="2019-01-01",frequency="Q")

In [ ]:

# 畫圖
plotter.plot(all_result_dict,index="portfolio_returns",N=0)
# 畫圖
plotter.plot(all_result_dict,index="portfolio_value",N=1)
# 列出DataFrame
plotter.table(all_result_dict,index="portfolio_value")

In [3]:
upper, middle, lower = data.indicator("BBANDS",timeperiod=20)

In [ ]:
upper

In [ ]:
pos = lower >middle

In [4]:
adx = data.indicator("ADX")

In [5]:
adx

,8905,8906,8908,8916,8917,8921,8923,8924,8926,8927,...,8932,8933,8935,8936,8937,8938,8940,8941,8942,8996
date,,,,,,,,,,,,,,,,,,,,,
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-27,43.064584,22.572775,36.242557,23.341766,40.416591,14.885186,14.439240,24.428500,62.927491,29.344066,...,7.268114,35.256796,24.109522,35.962776,18.864991,22.258428,30.096287,24.318052,30.235295,22.286484
2023-03-28,40.399614,21.236900,33.706960,23.615838,44.020672,15.003582,15.277090,23.319297,63.348618,28.213694,...,7.297675,36.817763,22.691014,33.982494,17.766580,21.510423,28.398922,22.673842,28.865202,20.827492
2023-03-29,38.260802,19.996444,32.080248,23.713304,47.367319,16.417918,16.055093,22.030720,63.226673,26.706934,...,7.256138,38.267232,21.373829,31.761336,17.569059,20.339863,26.964210,21.207109,27.373221,19.472714


In [6]:
adx = data.indicator("ADX", timperiod = 50)
adx

,8905,8906,8908,8916,8917,8921,8923,8924,8926,8927,...,8932,8933,8935,8936,8937,8938,8940,8941,8942,8996
date,,,,,,,,,,,,,,,,,,,,,
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-27,43.064584,22.572775,36.242557,23.341766,40.416591,14.885186,14.439240,24.428500,62.927491,29.344066,...,7.268114,35.256796,24.109522,35.962776,18.864991,22.258428,30.096287,24.318052,30.235295,22.286484
2023-03-28,40.399614,21.236900,33.706960,23.615838,44.020672,15.003582,15.277090,23.319297,63.348618,28.213694,...,7.297675,36.817763,22.691014,33.982494,17.766580,21.510423,28.398922,22.673842,28.865202,20.827492
2023-03-29,38.260802,19.996444,32.080248,23.713304,47.367319,16.417918,16.055093,22.030720,63.226673,26.706934,...,7.256138,38.267232,21.373829,31.761336,17.569059,20.339863,26.964210,21.207109,27.373221,19.472714
